In [1]:
import mdtraj as md
import os
import sys
import numpy as np
import scipy as sp
from scipy import optimize
from scipy.optimize import leastsq
import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns
import math
import itertools    
from numpy import log2, zeros, mean, var, sum, loadtxt, arange, \
                  array, cumsum, dot, transpose, diagonal, floor
from numpy.linalg import inv, lstsq
import pyblock
import pandas as pd

np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})


In [7]:
!pwd

/data/asn/tica/lig_47/dist


In [8]:
pdb = '/data/asn/biorxiv2021-6626290-no-water-glue/lig47.pdb'
rep0 = '/data/asn/biorxiv2021-6626290-no-water-glue/ligand_47_1.xtc'

# table,bonds = md.load(pdb).topology.to_dataframe()
# first = table[table["resSeq"]==121].to_numpy()
# first[:,0] = np.arange(1, 1+len(first))
# second = table[table["resSeq"]!=121].to_numpy()
# second[:,0] = np.arange(first[-1,0]+1, first[-1,0]+1 + len(second))
# final = np.concatenate([first, second], axis=0)
# df = pd.DataFrame(data = final, columns = table.columns)
# top_fix = md.Topology.from_dataframe(df, bonds)
#trj = md.load(rep0, top =top_fix)
trj = md.load(rep0, top =pdb)
trj.center_coordinates()
top = trj.topology
first_frame = 0
last_frame = trj.n_frames
n_frames = trj.n_frames

In [9]:
nres = []
for res in trj.topology.residues:
    nres.append(res.resSeq)
sequence = (' %s' % [residue for residue in trj.topology.residues])
resname = (' %s' % [residue.name for residue in trj.topology.residues])
resindex = (' %s' % [residue.index for residue in trj.topology.residues])
prot_top = top.subset(top.select('protein'))
prot_res = []
for res in prot_top.residues:
    prot_res.append(res.resSeq)
prot_resname = (' %s' % [residue.name for residue in prot_top.residues])
residues = len(set(prot_res))

#log = open("/Users/paulrobustelli/Desktop/Sa_calc.log", "w")
print("** SYSTEM INFO **\n")
print("Number of atoms: %d\n" % trj.n_atoms)
print("Number of residues: %d\n" % len(set(nres)))
print("Number of protein residues: %d\n" % len(set(prot_res)))
print("Number of frames: %d\n" % trj.n_frames)
print("Starting frame: %d\n" % first_frame)
print("Last frame: %d\n" % last_frame)
print("sequence: %s\n" % sequence)
print("residue names: %s\n" % resname)
print("residue index: %s\n" % resindex)


residue_offset = 0
prot_res_renum = np.asarray(prot_res)+residue_offset
residue_number = range(0, residues)
residue_number_offsetres = range(residue_offset, residue_offset+residues)



** SYSTEM INFO **

Number of atoms: 338

Number of residues: 21

Number of protein residues: 20

Number of frames: 1100889

Starting frame: 0

Last frame: 1100889

sequence:  [ASP121, ASN122, GLU123, ALA124, TYR125, GLU126, MET127, PRO128, SER129, GLU130, GLU131, GLY132, TYR133, GLN134, ASP135, TYR136, GLU137, PRO138, GLU139, ALA140, ASP121, <1>1]

residue names:  ['ASP', 'ASN', 'GLU', 'ALA', 'TYR', 'GLU', 'MET', 'PRO', 'SER', 'GLU', 'GLU', 'GLY', 'TYR', 'GLN', 'ASP', 'TYR', 'GLU', 'PRO', 'GLU', 'ALA', 'ASP', '<1>']

residue index:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]



In [7]:
def distance_matrix(sel1,sel2,offset1,offset2,traj,measure,periodic):
    ''' RETURNS: dmat,np.array(pairs),np.array(pairs_index),index,columns,len(sel1)..(x),len(sel2)....(y)
    '''
    pair_distances = []
    pairs = []
    pairs_index = []
    if measure == "residues":
        index = [traj.topology.residue(i) for i in sel1]
        columns = [traj.topology.residue(j) for j in sel2]
        for i in sel1:
            for j in sel2:
                pairs.append("{},{}".format(traj.topology.residue(i),traj.topology.residue(j)))
                pairs_index.append([i+offset1,j+offset2])
                if i==j:
                    dist = np.zeros(traj.n_frames)
                    pair_distances.append(dist)
                else:
                    dist = md.compute_contacts(traj,[[i,j]],periodic=periodic)[0][:,0]
                    pair_distances.append(dist)
    if measure == "atoms":
        index = [traj.topology.atom(i) for i in sel1]
        columns = [traj.topology.atom(j) for j in sel2]
        for i in sel1:
            for j in sel2:
                pairs.append("{},{}".format(traj.topology.atom(i),traj.topology.atom(j)))
                pairs_index.append([i+offset1,j+offset2])
                if i==j:
                    dist = np.zeros(traj.n_frames)
                    pair_distances.append(dist)
                else:
                    dist = md.compute_distances(traj,[[i,j]],periodic=periodic)[:,0]
                    pair_distances.append(dist)
    dist_feat_arr = np.stack(pair_distances,axis=1)
    return dist_feat_arr,np.array(pairs),np.array(pairs_index),index,columns,np.array([len(sel1),len(sel2)])
dmat, pairs, pairs_idx, index, col, xy = distance_matrix(np.arange(0,20),[20], 0,0,trj,"residues", True)
dmat_bin = np.where(dmat<.5,1,0)

In [4]:
np.save("distance_matrix_lig47.npy", dmat)

In [ ]:
#Select Ligand Charge Groups
#Ligand Charged atom is N-296
Ligand_Pos_Charges=[296]
Ligand_Neg_Charges=[]

def add_charge_pair(pairs,pos,neg,contact_prob):
    if pos not in pairs: 
        pairs[pos] = {} 
    if neg not in pairs[pos]:
        pairs[pos][neg] = {}
    pairs[pos][neg] = contact_prob

#Select Protein Charge Groups
#Add Apropriate HIS name if there is a charged HIE OR HIP in the structure 
Protein_Pos_Charges=top.select("(resname ARG and name CZ) or (resname LYS and name NZ) or (resname HIE and name NE2) or (resname HID and name ND1)")
#Protein_Neg_Charges=[]
Protein_Neg_Charges=top.select("(resname ASP and name CG) or (resname GLU and name CD) or (name OXT) or (resname NASP and name CG)")
neg_res=[]
pos_res=[]
                               
for i in Protein_Neg_Charges:
    neg_res.append(top.atom(i).residue.resSeq)

for i in Protein_Pos_Charges:
    pos_res.append(top.atom(i).residue.resSeq)                               
                               
print("Negatively Charged Residues:", neg_res)
print("Posiitively Charged Residues", pos_res)

charge_pairs_ligpos=[]                      
for i in Ligand_Pos_Charges:
    for j in Protein_Neg_Charges:              
        charge_pairs_ligpos.append([i,j])
        pos=top.atom(i)
        neg=top.atom(j) 

contact_1  = md.compute_distances(trj, charge_pairs_ligpos)
#contact_2  = md.compute_distances(trj, charge_pairs_ligneg)

In [ ]:
np.save("charge_matrix_lig47.npy", contact_1)

In [ ]:
# Calculate Hydrophobic contacts
ligand_hphob = top.select("resid 20 and type C")
protein_hphob = top.select("protein and element C")


ligand_hphob_atoms = []
for atom in ligand_hphob:
    ligand_hphob_atoms.append(top.atom(atom))

protein_hphob_atoms = []
for atom in protein_hphob:
    protein_hphob_atoms.append(top.atom(atom))

print(ligand_hphob)
print(ligand_hphob_atoms)

print(protein_hphob)
print(protein_hphob_atoms)


def add_contact_pair(pairs, a1, a2, a1_id, a2_id, prot_res, contact_prob):
    if prot_res not in pairs:
        pairs[prot_res] = {}
    if a2 not in pairs[prot_res]:
        pairs[prot_res][a2] = {}
    if a1_id not in pairs[prot_res][a2]:
        pairs[prot_res][a2][a1_id] = contact_prob


hphob_pairs = []
for j in ligand_hphob:
    for i in protein_hphob:
        hphob_pairs.append([i, j])


contact = md.compute_distances(trj, hphob_pairs)
contacts = np.asarray(contact).astype(float)

In [ ]:
np.save("hphob_matrix_lig47.npy", contacts)

In [ ]:
# Definie Hydrogen Bond Donors in EPI-002
lig_hbond_donors = [[329, 331], [329, 330]]
hbonds = print_donors_acceptors(
    trj[0], angle_cutoff=150, distance_cutoff=0.35, lig_donor_index=lig_hbond_donors)
dist = md.compute_distances(trj, hbonds[:, [0,2]], periodic=False)

In [ ]:
np.save("hbond_matrix_lig47.npy", dist)